Panel lets you add interactive controls for just about anything you can display in Python. Panel can help you build simple interactive apps, complex multi-page dashboards, or anything in between. As a simple example, let's say you have created a function to plot a sine wave using Matplotlib and the Pandas `.plot()` command:

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
%matplotlib inline

def mplplot(df, **kwargs):
    fig = df.plot().get_figure()
    plt.close(fig)
    return fig

def sine(frequency=1.0, amplitude=1.0, n=200, view_fn=mplplot):
    xs = np.arange(n)/n*20.0
    ys = amplitude*np.sin(frequency*xs)
    df = pd.DataFrame(dict(y=ys), index=xs)
    return view_fn(df, frequency=frequency, amplitude=amplitude, n=n)

sine(1.5, 2.5)

If we wanted to try out lots of combinations of these values to understand how frequency and amplitude affect this plot, we could reevaluate the above cell lots of times, but that would be a slow and painful process, and is only really appropriate for users who are comfortable with editing Python code. 

## Interactive Panels

Instead of editing code, it's much quicker and more straightforward to use sliders to adjust the values interactively.  You can easily make a Panel app to explore a function's parameters using `pn.interact`, which is similar to the [ipywidgets interact function](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html):

In [ ]:
import panel as pn
pn.extension()

pn.interact(sine)

As long as you have a live Python process running, dragging these widgets will trigger a call to the `sine` callback function, evaluating it for whatever combination of parameter values you select and displaying the results. A Panel like this makes it very easy to explore any function that produces a visual result of a [supported type](https://github.com/pyviz/panel/issues/2), such as Matplotlib (as above), Bokeh, Plotly, Altair, or various text and image types.

## Components of Panels

`interact` is convenient, but what if you want more control over how it looks or works?  First, let's see what `interact` actually creates, by grabbing that object and displaying its representation:

In [ ]:
i = pn.interact(sine, n=(5,100))
i.pprint()

As you can see, the `interact` call created a `pn.Column` object consisting of a WidgetBox (with 3 widgets) and a `pn.Row` with one Matplotlib figure object.  Panel is compositional, so you can mix and match these components any way you like, adding other objects as needed:

In [ ]:
pn.Row(i[1][0], pn.Column("<br>\n# Sine waves", i[0][0], i[0][1]))

Note that the widgets stay linked to their plot even if they are in a different notebook cell:

In [ ]:
i[0][2]

Also note that Panel widgets are reactive, so they will update even if you set the values by hand:

In [ ]:
i[0][0].value = 2.5

## Composing new Panels

You can use this compositional approach to combine different components such as widgets, plots, text, and other elements needed for an app or dashboard in arbitrary ways. The ``interact`` example builds on a reactive programming model, where an input to the function changes and Panel reactively updates the output of the function. ``interact`` is a convenient way to create widgets from the arguments to your function automatically, but Panel also provides a more explicit reactive API letting you specifically define connections between widgets and function arguments, and then lets you compose the resulting dashboard manually from scratch.

In the example below we explicitly declare each of the components of an app: widgets, a function to compute sine values, column and row containers, and the completed `sine_panel` app. Widget objects have multiple "parameters" (current value, allowed ranges, and so on), and here we will use Panel's ``depends`` decorator to declare that function's input values should come from the widgets' ``value`` parameters. Now when the function and the widgets are displayed, Panel will automatically update the displayed output whenever any of the inputs change:

In [ ]:
import panel.widgets as pnw

frequency  = pnw.FloatSlider(name='frequency', value=1, start=1.0, end=5)
amplitude  = pnw.FloatSlider(name='amplitude', value=1, start=0.1, end=10)

@pn.depends(frequency.param.value, amplitude.param.value)
def reactive_sine(frequency, amplitude):
    return sine(frequency, amplitude)

widgets    = pn.Column("<br>\n# Sine waves", frequency, amplitude)
sine_panel = pn.Row(reactive_sine, widgets)

sine_panel

## Deploying Panels

The above panels all work in the notebook cell (if you have a live Jupyter kernel running), but unlike other approaches such as ipywidgets, Panel apps work just the same in a standalone server. For instance, the app above can be launched as its own web server on your machine by uncommenting and running the following cell:

In [ ]:
#sine_panel.show()

Or, you can simply mark whatever you want to be in the separate web page with `.servable()`, and then run the shell command `panel serve --show Introduction.ipynb` to launch a server containing that object. (Here, we've also added a semicolon to avoid getting another copy of the sine_panel here in the notebook.)

In [ ]:
sine_panel.servable();

## Declarative Panels

The above compositional approach is very flexible, but it ties your domain-specific code (the parts about sine waves) with your widget display code. That's fine for small, quick projects or projects dominated by visualization code, but what about large-scale, long-lived projects, where the code is used in many different contexts over time, such as in large batch runs, one-off command-line usage, notebooks, and deployed dashboards?  For larger projects like that, it's important to be able to separate the parts of the code that are about the underlying domain (i.e. application or research area) from those that are tied to specific display technologies (such as Jupyter notebooks or web servers). 

For such usages, Panel supports objects declared with the separate [Param](http://param.pyviz.org) library, which provides a GUI-independent way of capturing and declaring the parameters of your objects (and dependencies between your code and those parameters), in a way that's independent of any particular application or dashboard technology. For instance, the above code can be captured in an object that declares the ranges and values of all parameters, as well as how to generate the plot, independently of the Panel library or any other way of interacting with the object:

In [ ]:
import param

class Sine(param.Parameterized):
    amplitude = param.Number(default=1, bounds=(0, None), softbounds=(0,5))
    frequency = param.Number(default=2, bounds=(0, 10))
    n = param.Integer(default=200, bounds=(1, 200))

    def view(self):
        return sine(self.frequency, self.amplitude, self.n)
    
sine_obj = Sine()

The `Sine` class and the `sine_obj` instance have no dependency on Panel, jupyter, or any other GUI or web toolkit; they simply declare facts about a certain domain (such as that sine waves take frequency and amplitude parameters, and that amplitude is a number greater than or equal to zero).  This information is then enough for Panel to create an editable and viewable representation for this object without having to specify anything that depends on the domain-specific details encapsulated in `sine_obj`:

In [ ]:
pn.Row(sine_obj.param, sine_obj.view)

To support a particular domain, you can create hierarchies of such classes encapsulating all the parameters and functionality you need across different families of objects, with both parameters and code inheriting across the classes as appropriate, all without any dependency on a particular GUI library or even the presence of a GUI at all.  This approach makes it practical to maintain a large codebase, all fully displayable and editable with Panel, in a way that can be maintained and adapted over time.

## Linking plots and actions between panes

The above approaches each work with a very wide variety of displayable objects, including images, equations, tables, and plots. In each case, Panel provides interactive functionality using widgets and updates the displayed objects accordingly, while making very few assumptions about what actually is being displayed. Panel also supports richer, more dynamic interactivity where the displayed object is itself interactive, such as the JavaScript-based plots from  Bokeh and Plotly.

For instance, if we substitute the [Bokeh](http://bokeh.pydata.org) wrapper [hvPlot](http://hvplot.pyviz.org) for the Matplotlib wrapper provided with Pandas, we automatically get interactive plots that allow zooming, panning and hovering:

In [ ]:
import hvplot.pandas

def hvplot(df, **kwargs):
    return df.hvplot()

pn.interact(sine, view_fn=hvplot)

These interactive actions can be combined with more complex interactions with a plot (e.g. tap, hover) to make it easy to explore data more deeply and uncover connections. For instance, we can use HoloViews to make a more full-featured version of the hvPlot example that dynamically updates to indicate the position on the unit circle when hovering over the sine curve:

In [ ]:
import holoviews as hv

tap = hv.streams.PointerX(x=0)

def hvplot2(df, frequency, **kwargs):
    plot = df.hvplot(width=500, padding=(0, 0.1))
    tap.source = plot

    def unit_circle(x):
        cx       = np.cos(x*frequency)
        sx       = np.sin(x*frequency)
        circle   = hv.Path([ hv.Ellipse(0,0,2),  [(-1,0), (1,0)],  [(0,-1), (0,1)]]).opts(color='black')
        triangle = hv.Path([ [(0,0), (cx,sx)],   [(0,0), (cx,0)],  [(cx,0), (cx,sx)]]).opts(color='red', line_width=2)
        labels   = hv.Labels([(cx/2, 0, '%.2f' % cx), (cx, sx/2., '%.2f' % sx)])
        labels   = labels.opts(padding=0.1, xaxis=None, yaxis=None, text_baseline='bottom')
        return (circle * triangle * labels)

    vline  = hv.DynamicMap(hv.VLine,    streams=[tap]).opts(color='black')
    circle = hv.DynamicMap(unit_circle, streams=[tap])
    
    return (plot * vline + circle).opts(toolbar="right")

unit_curve = pn.interact(sine, view_fn=hvplot2, n=(1, 200), frequency=(0, 10.))

pn.Column(pn.Row('# The Unit Circle', pn.Spacer(width=45), unit_curve[0][0], unit_curve[0][2]),
          unit_curve[1])

## Exploring further

As you have seen above, Panel can be used in several different ways, each appropriate for different applications:

- [Interact](Interact.ipynb): Instant GUI, given a function with arguments
- [Widgets](Widgets.ipynb): Explicitly instantiating widgets and linking them to actions
- [Parameters](Parameters.ipynb): Capturing parameters and their links to actions declaratively

Just pick the style that seems most appropriate for the task you want to do, then study that section of the user guide. Regardless of which approach you take, you'll want to learn more about Panel's panes and layouts:

- [Components](Components.ipynb): An overview of the core components of Panel including Panes, Widgets and Layouts
- [Customization](Customization.ipynb): How to set styles and sizes of Panel components
- [Deploy & Export](Deploy_and_Export.ipynb): An overview on how to display, export and deploy Panel apps and dashboards

Finally, if you are building a complex multi-stage application, you can consider our preliminary support for organizing and navigating between pages (still a work in progress!):

- [Pipelines](Pipelines.ipynb): Making multi-stage processing pipelines in notebooks and as deployed apps